# Experience of paper *Parallel Higher Order Alternating Least Square paper*

Small e-commerce dataset

Model code has been updated since the paper publication which can explain differences

In [1]:
from RecommenderSystem.HOALS import HOALS
from RecommenderSystem.metrics import rank_measure
from sktensor.dtensor import dtensor
from sktensor.tucker import hosvd # baseline
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../../data/smallEcommerce.csv',sep=";")
data.head()

,fullVisitorId,codssfamille,nPPview,nATC,nConv
0,7926108618558685613,121114,2,0,0
1,8994729742035347686,131311,2,0,0
2,3751682749408060125,124211,2,0,0
3,4070221717608550090,242111,7,2,1
4,1924054936669126100,246214,1,1,0


In [3]:
nUser = len(set(data['fullVisitorId']))
nItem = len(set(data['codssfamille']))

userPivot = pd.DataFrame({'fullVisitorId':list(set(data['fullVisitorId'])),
                         'user':range(nUser)})
itemPivot = pd.DataFrame({'codssfamille':list(set(data['codssfamille'])),
                         'item':range(nItem)})

data = pd.merge(pd.merge(data,userPivot),itemPivot)
del data['fullVisitorId']
del data['codssfamille']
data.head()

,nPPview,nATC,nConv,user,item
0,2,0,0,274,75
1,2,0,0,160,75
2,6,1,1,936,75
3,2,2,0,719,75
4,27,19,0,815,75


In [4]:
actions = ['nPPview','nATC','nConv']
df = pd.DataFrame(columns=['user','item','action','rating'])

for action in actions:
    ind = actions.index(action)
    tmp = data[['user','item',action]]
    tmp = tmp.rename(columns={action:'rating'})
    tmp = tmp.loc[tmp['rating']!=0,]
    tmp['action'] = ind
    df = df.append(tmp)
df.head()
del data

** preprocessing **

In [ ]:
data.loc[data['nPPview']>50] = 50

In [5]:
threshold = 0.7
df['cv'] = np.random.random(len(df))

trainingData = df.loc[df['cv']<threshold,['user','item','action','rating']]
testData = df.loc[df['cv']>=threshold,['user','item','action','rating']]

## Train HOALS

** cross validation parameters **

In [6]:
models = ['tucker','parafac']
lbdas = [0.01,0.1,1.]
implicitPrefs = [True,False]
alphas = [0.0,0.5,1.0]
rankss = [[10,200,2],[50,50,2]]

In [16]:
sc.setCheckpointDir("checkpoints")
hoals = HOALS(ranks=[10,200,2],model='tucker',lbda=0.8,alpha=0.0,max_iter=10,
             dimensions_col=['user','item','action'],rating_col=['rating'],
             implicitPrefs=False)
hoals.fit(trainingData,True)

Start user Learning
time : 1.5179917812347412
Start item Learning
time : 47.475234270095825
Start action Learning
time : 1.4844989776611328
longest mode time : 47.475234270095825


** Compute rank measure **

In [17]:
indexes = testData[['user','item','action']].values.tolist()
r_hat = hoals.predict(indexes)
testData['r_hat_hoals'] = r_hat

rank_perf_hoals = dict.fromkeys(range(len(actions)))
for action in range(len(actions)):
    testData_action = testData.loc[testData['action']==action,]
    rs = list(testData_action.sort_values(by='r_hat_hoals',ascending=False).groupby('user')['rating'].apply(list))
    rank_perf_hoals[action] = np.round(100*rank_measure(rs),2)
    print("Rank measure of action",action,":",rank_perf_hoals[action],"%")

Rank measure of action 0 : 33.66 %
Rank measure of action 1 : 37.64 %
Rank measure of action 2 : 40.63 %


Rank measure of action 0 : 34.41 %
Rank measure of action 1 : 37.9 %
Rank measure of action 2 : 40.75 %
